# Langchain Agent 101

This is the first module for trying Langchain agent. In this exercise, we will do following things

1. Create an agent
2. Add tool to access internet with search capability and give answer on any topic.


**Step 1**

Install langchain related dependency

In [ ]:
!pip install langchain_openai
!pip install langchain_core
!pip install langchain.agents
!pip install langchain.tools
!pip install duckduckgo-search

ERROR: Could not find a version that satisfies the requirement langchain.agents (from versions: none)
ERROR: No matching distribution found for langchain.agents


In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = "my_key" ## import this key from Google secret manager
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# Create the ReAct template
from langchain_core.prompts import PromptTemplate

react_template = """ Answer the following question as best as you can. You have access to the following tools:

{tools}

Use the following format:

Question : The input question you must answer to
Thought : you should always think about what to do
Action: The action to take, should be on e of [{tool_names}]
Action Input : the input to the action
Observation : the result of the action
.... (this thought/Action/Action input/Observation can repeat N times)
Thought: I know the final answer
Final Answer: The final answer to the original question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate(template = react_template, input_variables=["tools", "tool_names", "input", "agent_scratchpad"])


In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# One example of passing tools to the agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name = "DuckDuck search tool",
    description="A web search engine. Use this to seach engine for a general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
!pip install -U duckduckgo_search==5.3.1b1

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD and then convert it to EUR using the given exchange rate.
Action: DuckDuck search tool
Action Input: "current price of MacBook Pro in USD"

DuckDuckGoSearchException: _text_extract_json() keywords='current price of MacBook Pro in USD' ValueError: subsection not found